In [1]:
import gc
import glob
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import utils
import utils_fe
from gbm_pipeline import GBMPipeline
from scipy.stats import mstats
from tqdm import tqdm

from sklearn.metrics import roc_auc_score

/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


Using TensorFlow backend.


In [2]:
src_oof = 'oof/'
src_preds = '/home/w/Projects/Toxic/scripts/predictions/'
src_features = '../data/features/'

run_kfold = True
split_for_validation = True
add_features = True

target_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [33]:
train, test = utils.load_data('../data/', 'BasicClean2')

oof_tr, oof_te = utils.load_predictions(src_oof, load_oof=True, contains='', contains2=None)
oof_tr, oof_te = np.asarray(oof_tr), np.asarray(oof_te)
print(oof_tr.shape, oof_te.shape)

oof_tr = oof_tr[[1, 5, -1]]
oof_te = oof_te[[1, 5, -1]]

X_train_oof = pd.DataFrame()
X_test_oof = pd.DataFrame()

for i in range(oof_tr.shape[0]):
    oof_tr_part = oof_tr[i][:, :, 0]
    oof_te_part = oof_te[i].mean(axis=-1)[:, :]
    X_train_oof = pd.concat([X_train_oof, pd.DataFrame(oof_tr_part)], axis=1)
    X_test_oof = pd.concat([X_test_oof, pd.DataFrame(oof_te_part)], axis=1)


f_cols = []

for i in range((X_train_oof.shape[1] // 6)):
    f_cols.extend(['{}_preds_model{}'.format(target_columns[x], i) for x in range(len(target_columns))])

X_train_oof.columns = f_cols
X_test_oof.columns = f_cols

Load data with basic cleaning with non-alphanumeric contained.

Train files:

['oof/train/Conv2Dmodel128_KF3_KFold1_FastText2_BasicClean2_300dim_200k_320len_random0_5fold_BS256_Nadam0.001_Stratified_27.31593.pkl',
 'oof/train/Conv2Dmodel128_KF3_KFold1_FastText2_BasicClean2_300dim_200k_320len_random0_5fold_BS256_Nadam0.001_Stratified_fromCheck_27.31593.pkl',
 'oof/train/GRUCapsuleBidirectional128_KF2_KFold1_FastText2_BasicClean2_300dim_200k_320len_random0_5fold_BS256_Adam0.001_Stratified_22.37225.pkl',
 'oof/train/GRUConvdeepBidirectional128_KF3_KFold1_FastText2_BasicClean2_300dim_200k_320len_random0_5fold_BS256_Adam0.001_Stratified_20.75529.pkl',
 'oof/train/GRUConvdeepBidirectional128_KF3_KFold1_FastText2_BasicClean2_300dim_200k_320len_random0_5fold_BS256_Adam0.001_Stratified_fromCheck_20.75529.pkl',
 'oof/train/GRUHierarchicalBidirectional128_KF3_KFold1_FastText2_BasicClean2_300dim_200k_320len_random0_5fold_BS256_Adam0.001_Stratified_23.81552.pkl',
 'oof/train/GRUHierarchicalBidirect

In [34]:
toxic_cols = utils.pick_target_columns(X_train_oof, target_columns)
severe_toxic_cols = utils.pick_target_columns(X_train_oof, target_columns, col='severe_toxic')
obscene_cols = utils.pick_target_columns(X_train_oof, target_columns, col='obscene')
threat_cols = utils.pick_target_columns(X_train_oof, target_columns, col='threat')
insult_cols = utils.pick_target_columns(X_train_oof, target_columns, col='insult')
identity_cols = utils.pick_target_columns(X_train_oof, target_columns, col='identity_hate')

In [35]:
use_geo_mean = True


if use_geo_mean:
    print('Using geometric mean for averaging.')
    train_toxic_geo = mstats.gmean(X_train_oof[toxic_cols], axis=1)
    train_severe_toxic_geo = mstats.gmean(X_train_oof[severe_toxic_cols], axis=1)
    train_obscene_cols_geo = mstats.gmean(X_train_oof[obscene_cols], axis=1)
    train_threat_geo = mstats.gmean(X_train_oof[threat_cols], axis=1)
    train_insult_geo = mstats.gmean(X_train_oof[insult_cols], axis=1)
    train_identity_geo = mstats.gmean(X_train_oof[identity_cols], axis=1)

    test_toxic_geo = mstats.gmean(X_test_oof[toxic_cols], axis=1)
    test_severe_toxic_geo = mstats.gmean(X_test_oof[severe_toxic_cols], axis=1)
    test_obscene_cols_geo = mstats.gmean(X_test_oof[obscene_cols], axis=1)
    test_threat_geo = mstats.gmean(X_test_oof[threat_cols], axis=1)
    test_insult_geo = mstats.gmean(X_test_oof[insult_cols], axis=1)
    test_identity_geo = mstats.gmean(X_test_oof[identity_cols], axis=1)
    
else:
    print('Using harmonic mean for averaging.')
    train_toxic_geo = mstats.hmean(X_train_oof[toxic_cols], axis=1)
    train_severe_toxic_geo = mstats.hmean(X_train_oof[severe_toxic_cols], axis=1)
    train_obscene_cols_geo = mstats.hmean(X_train_oof[obscene_cols], axis=1)
    train_threat_geo = mstats.hmean(X_train_oof[threat_cols], axis=1)
    train_insult_geo = mstats.hmean(X_train_oof[insult_cols], axis=1)
    train_identity_geo = mstats.hmean(X_train_oof[identity_cols], axis=1)

    test_toxic_geo = mstats.hmean(X_test_oof[toxic_cols], axis=1)
    test_severe_toxic_geo = mstats.hmean(X_test_oof[severe_toxic_cols], axis=1)
    test_obscene_cols_geo = mstats.hmean(X_test_oof[obscene_cols], axis=1)
    test_threat_geo = mstats.hmean(X_test_oof[threat_cols], axis=1)
    test_insult_geo = mstats.hmean(X_test_oof[insult_cols], axis=1)
    test_identity_geo = mstats.hmean(X_test_oof[identity_cols], axis=1)
    
    
oof_test_mean = np.vstack([test_toxic_geo, test_severe_toxic_geo, test_obscene_cols_geo,
                                 test_threat_geo, test_insult_geo, test_identity_geo]).swapaxes(0, 1)
oof_train_mean = np.vstack([train_toxic_geo, train_severe_toxic_geo, train_obscene_cols_geo,
                                 train_threat_geo, train_insult_geo, train_identity_geo]).swapaxes(0, 1)

print(oof_train_mean.shape)
# print(oof_train_mean[:10, :])

print(oof_test_mean.shape)
# print(oof_test_mean[:10, :])

oof_roc = roc_auc_score(train[target_columns].values, oof_train_mean)
print('OOF train ROC score: {}'.format(oof_roc))

Using geometric mean for averaging.
(159571, 6)
(153164, 6)
OOF train ROC score: 0.9880452464923707


In [36]:
run_name = 'AllGeometricMean_5fold_ChecksOnly'

submission = utils.output_submission(
    oof_test_mean, run_name, save=True)

In [56]:
sub_src = '/home/w/Projects/Toxic/submissions/'

s1 = pd.read_csv(sub_src + "7ModelsGeometricMean_5fold_ChecksOnly.csv")
s2 = pd.read_csv(sub_src + "blend_it_all_b2_current_best_pb.csv")
s3 = pd.read_csv(sub_src + "('LGB_KFold_StackerNext_Params1',)_6_loss_0.98833.csv")

s = s1.copy()
s.iloc[:, 1:] = (s1.iloc[:, 1:] * 2 + s2.iloc[:, 1:] * 8) / 10.

In [57]:
s

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.966197,0.433600,0.940244,0.236431,0.893872,0.478776
1,0000247867823ef7,0.104245,0.077891,0.093681,0.082839,0.092530,0.083731
2,00013b17ad220c46,0.105542,0.084247,0.096837,0.087563,0.092979,0.090283
3,00017563c3f7919a,0.090521,0.068316,0.083436,0.083919,0.079560,0.072656
4,00017695ad8997eb,0.124771,0.085620,0.104455,0.094517,0.099240,0.091085
5,0001ea8717f6de06,0.093975,0.071496,0.085036,0.086207,0.085752,0.078351
6,00024115d4cbde0f,0.119452,0.076916,0.100202,0.087994,0.101504,0.084396
7,000247e83dcc1211,0.563669,0.114680,0.169538,0.112640,0.208917,0.122744
8,00025358d4737918,0.183315,0.085789,0.122589,0.089026,0.140204,0.098938
9,00026d1092fe71cc,0.102586,0.075822,0.089489,0.086627,0.087258,0.083707


In [58]:
s.to_csv(sub_src + 'BlendOfBlend_7Models_and_CurrentPublicLBBest_Mine02.csv', index=False)